<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-gradio-rag/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main Notebook
Build a RAG pipeline from a file!

## Documentation
https://docs.google.com/document/d/1iRPcqsYZj0Jmd6QqI6UoT2mVbs3BjGkpIpGCI5SFQCM/edit?tab=t.0

In [ ]:
# Install dependencies:
!pip install langchain openai pinecone-client gradio pypdf tiktoken

In [ ]:
# Imports
import os
import gradio as gr
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import pinecone
from google.colab import userdata

In [ ]:
# Set API keys
os.environ["OPENAI_API_KEY"] =  userdata.get("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] =  userdata.get("PINECONE_API_KEY")
os.environ["PINECONE_ENVIRONMENT"] =  userdata.get("PINECONE_ENVIRONMENT")

In [ ]:
# Initialize Pinecone
pinecone.init(api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENVIRONMENT"])
index_name = "rag-demo"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536)  # matches OpenAI embedding size

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
# Function to process uploaded document
def process_document(file):
    # Load PDF or text
    if file.name.endswith(".pdf"):
        loader = PyPDFLoader(file.name)
        docs = loader.load()
    else:
        text = open(file.name, "r", encoding="utf-8").read()
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        docs = splitter.create_documents([text])

    # Store in Pinecone
    vectorstore = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)
    return "Document uploaded and indexed successfully!"

In [ ]:
# Function to answer questions
def answer_question(query):
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    llm = ChatOpenAI(model="gpt-4", temperature=0)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")
    return qa_chain.run(query)

In [ ]:
# Build a Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 📘 RAG Q&A Demo with Document Upload")
    with gr.Row():
        upload = gr.File(label="Upload a PDF or TXT file")
        upload_output = gr.Textbox(label="Upload Status")
    upload.upload(process_document, upload, upload_output)
    query = gr.Textbox(lines=2, placeholder="Ask a question...")
    answer = gr.Textbox(label="Answer")
    query.submit(answer_question, query, answer)
demo.launch()

#